In [1]:
import os
import json
import re

input_ = open('/home/xuwenshen/data/big_data/raw_cases')
output_ = open('/home/xuwenshen/data/big_data/2017_3_13/cases', 'w')


pattern_symbol = re.compile(r'[。！；;（）]')
pattern_defend = re.compile(r'被告|罪犯|犯|罪|非法|涉嫌')

counter = 0
while True:
    
    line = input_.readline()
    if line == '':
        break
    line = json.loads(line)
    
    #reason begin
    if len(line['reason']) > 250: continue
    #reason end 
    
    # source concate begin
    source = ''
    for i in range(len(line['source'])):
        source += line['source'][i]
    if len(source) > 1500: continue
    # source concate end
    
    source_tmp = re.sub(pattern_symbol, '，', source)
    source_tmp = source_tmp.split('，')
    
    # search defendant information begin
    source_def = ''
    defendant_candidate = []
    for i in range(len(source_tmp)):
        if re.search(pattern_defend, source_tmp[i]):
            if len(source_tmp[i]) < 30:
                source_def += source_tmp[i] + '，'
            else:
                defendant_candidate.append(source_tmp[i])
    
    for i in range(len(defendant_candidate)):
        source_def += defendant_candidate[i] + '，'
            
    if len(source_def) > 300: continue
          
    # search defendant information end 
    
    to_write = {'source':source, 'reason': line['reason'], 'defendant': source_def}
    
    output_.write(json.dumps(to_write) + '\n')
        
    counter += 1

input_.close()
output_.close()
print (counter)

1725672


In [2]:
import os
import json
from tqdm import *

input_ = open('/home/xuwenshen/data/big_data/2017_3_13/cases')

raw_words = dict()


for j in tqdm(range(1725672)):
    line = input_.readline()
    if line == '':
        break
    line = json.loads(line)
    for i in range(len(line['source'])):
        if line['source'][i] in raw_words:
            raw_words[line['source'][i]] += 1
        else:
            raw_words.update({line['source'][i]:1})
                
    
    for i in range(len(line['reason'])):
        if line['reason'][i] in raw_words:
            raw_words[line['reason'][i]] += 1
        else:
            raw_words.update({line['reason'][i]:1})
    
input_.close()

print (len(raw_words))


100%|██████████| 1725672/1725672 [10:13<00:00, 2813.25it/s]

8149


In [3]:
output_ =open('/home/xuwenshen/data/big_data/2017_3_13/tot_words', 'w')

for word in raw_words:
    output_.write(word + ' ' + str(raw_words[word]))
    output_.write('\n')
output_.close()


print ('done')

done


In [3]:
input_ = open('/home/xuwenshen/data/big_data/2017_3_13/tot_words')
words = dict()


while True:
    line = input_.readline()
    if line == '':
        break
        
    line = line.rstrip('\n').split()
    if len(line) != 2:
        continue
    words.update({line[0]:int(line[1])})

input_.close()

sorted_words = sorted(words.items(), key = lambda words:words[1])


In [5]:
hwords = set()

for i in range(len(sorted_words)):
    if sorted_words[i][1] >= 5000:
        hwords.add(sorted_words[i][0])


hwords.add('eos')
hwords.add('pad')
hwords.add('go')



2184


In [6]:
output_ = open('/home/xuwenshen/data/big_data/2017_3_13/words', 'w')

for word in hwords:
    output_.write(word+'\n')

output_.close()

print ('done')



done


In [7]:
import json
from tqdm import *

input_ = open('/home/xuwenshen/data/big_data/2017_3_13/cases')
output_ = open('/home/xuwenshen/data/big_data/2017_3_13/simplified_unshuffled_cases', 'w')

for j in tqdm(range(1725672)):
    
    line = input_.readline()
    if line == '':
        break
    line = json.loads(line)

    new_text = dict()
    
    source_tx = ''
    for i in range(len(line['source'])):
        if line['source'][i] in hwords:
            source_tx += line['source'][i]
    new_text.update({'source':source_tx})
    
    defendant_tx = ''
    for i in range(len(line['defendant'])):
        if line['defendant'][i] in hwords:
            defendant_tx += line['defendant'][i]
    new_text.update({'defendant':defendant_tx})
    
    reason_tx = ''
    for i in range(len(line['reason'])):
        if line['reason'][i] in hwords:
            reason_tx += line['reason'][i]
    new_text.update({'reason':reason_tx})
    
    output_.write(json.dumps(new_text))
    output_.write('\n')

input_.close()
output_.close()
print ('done')

100%|██████████| 1725672/1725672 [13:01<00:00, 2209.57it/s]


done


In [8]:
import random
from tqdm import *

input_ = open('/home/xuwenshen/data/big_data/2017_3_13/simplified_unshuffled_cases')

datas = [_.rstrip('\n') for _ in tqdm(input_.readlines())]

random.shuffle(datas)

input_.close()

output_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_cases', 'w')

for i in tqdm(range(len(datas))):
    output_.write(datas[i])
    output_.write('\n')

output_.close()

print (len(datas))
print ('done')

100%|██████████| 1725672/1725672 [00:43<00:00, 39477.63it/s]


1725672
done


In [9]:
import random
from tqdm import *

input_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_cases')

output_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_train', 'w')

for i in tqdm(range(1600000)):
    line = input_.readline()
    output_.write(line)

output_.close()

output_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_test', 'w')

for i in range(1600000, 1725672):
    line = input_.readline()
    output_.write(line)

output_.close()
input_.close()

print ('done')

100%|██████████| 1600000/1600000 [03:53<00:00, 6852.32it/s] 


done


In [11]:
import json
import re


input_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_test')

output_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_valid', 'w')

for i in tqdm(range(10000)):
    output_.write(input_.readline())

output_.close()

output_ = open('/home/xuwenshen/data/big_data/2017_3_13/shuffled_test1', 'w')

for i in tqdm(range(10000, 125672)):
    output_.write(input_.readline())

output_.close()
input_.close()


100%|██████████| 115672/115672 [00:01<00:00, 78454.18it/s]
